# <span style="color:green">2. Analyse des corrélations</span>

L’objectif de cette étape est d’analyser les relations entre les variables après discrétisation afin de palier au problème de **multi-colinéarité** et préparer correctement les données pour les étapes suivantes du projet.

Cette étape utilise **exclusivement** le dataset issu de l’étape précédente :
**credit_risk_dataset_discretized_final.csv**

Il contient :
- Les **11 variables discrétisées** (ChiMerge, Monotone, KMeans)
- La variable cible : `loan_status`

 Pour vérifier la corrélation entre les variables discrétisées, et entre les variables explicatives et la variable à expliquer, on effectue le **test d’indépendance du X²**.
Le test d’indépendance du X² consiste à déterminer si la valeur observée d’une variable dépend de la valeur observée d’une autre variable. On visualise aussi avec la matrice de corrélation de Cramer.

#### <span style="color:orange">Librairies</span>

In [1]:
import sys
import pandas as pd
import plotly.express as px

sys.path.append("../src")
from utils import chi2_independence_tests, cramers_v_matrix

#### <span style="color:orange">Chargement du dataset</span> 

In [2]:
# 1. Chargement et Préparation
df = pd.read_csv('../datasets/processed/credit_risk_dataset_discretized_final.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32572 entries, 0 to 32571
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   person_age                  32572 non-null  int64
 1   person_income               32572 non-null  int64
 2   person_home_ownership       32572 non-null  int64
 3   person_emp_length           32572 non-null  int64
 4   loan_intent                 32572 non-null  int64
 5   loan_grade                  32572 non-null  int64
 6   loan_amnt                   32572 non-null  int64
 7   loan_int_rate               32572 non-null  int64
 8   loan_status                 32572 non-null  int64
 9   loan_percent_income         32572 non-null  int64
 10  cb_person_default_on_file   32572 non-null  int64
 11  cb_person_cred_hist_length  32572 non-null  int64
dtypes: int64(12)
memory usage: 3.0 MB


In [3]:
# Séparation Features / Target
X = df.drop(columns=['loan_status'])
y = df['loan_status']

#### <span style="color:orange">Test d'indépendance de Chi-2</span>

Le test d’indépendance du χ² sert à déterminer si deux variables qualitatives (ou discrétisées) sont statistiquement indépendantes ou associées.

Hypothèses statistiques

Pour deux variables X1 et X2 :
- Hypothèse nulle : X1 et X2 sont indépendantes
- Hypothèse alternative X1 et X2 sont dépendantes 

In [4]:
test_chi2 = chi2_independence_tests(df.drop(columns=['loan_status', 'loan_grade']), alpha=0.05)
display(test_chi2.head())

,Variable 1,Variable 2,Chi-2,ddl,p_value,dependance_significative,chi2_valide
0,person_age,cb_person_cred_hist_length,39883.622942,6,0.0,True,True
1,loan_int_rate,cb_person_default_on_file,7440.067091,6,0.0,True,True
2,loan_amnt,loan_percent_income,6966.772340,9,0.0,True,True
3,person_income,loan_amnt,4318.562317,15,0.0,True,True
4,person_income,person_home_ownership,3885.711473,10,0.0,True,True


Ces couples de variables présentent une dépendance statistiquement significative au seuil de 5%. 

In [5]:
test_chi2.to_excel("../result/test_chi2.xlsx", index=False)

#### <span style="color:orange">Matrice de corrélation avec V de Cramer</span>

Cramér's V est une mesure de l’association entre deux variables qualitatives (catégorielles). C'est l'équivalent du coefficient de corrélation de Pearson mais pour des catégories. Il utilise la statistique du test du chi² pour l’indépendance de deux variables. 
- 0≤V≤1 : PLus V tend vers 0 et moins les variables catégorielles sont liées



In [6]:
V_matrix = cramers_v_matrix(df).round(2)

# Visualisation avec Plotly
fig = px.imshow(
    V_matrix,
    text_auto=True,
    color_continuous_scale='Viridis',
    title="Matrice de corrélation Cramer's V",
    labels=dict(x="Variables", y="Variables", color="Cramér's V")
)
fig.update_layout(
    xaxis_side="top",
    width=800,
    height=800
)
fig.show()

Ces observations sont bien en adéquation avec les tets d'indépendance de Chi-2 et sont à prendre en compte pour la suite de l'analyse.
Les variables `person_income`,`loan_grade`,`cb_person_default_on_file` et `person_home_ownership` figurent parmi les plus correlées à notre target `loan_status`.

#### <span style="color:orange">Prochaine étape: La sélection de variables</span>